In [5]:
import pandas as pd
import re
from glob import glob
from tqdm.notebook import tqdm

In [6]:
files = glob('./data/*.csv')

In [7]:
result = pd.DataFrame(columns=['title','contents'])
for ifile in files:
    df = pd.read_csv(ifile)
    result = pd.concat([result,df], axis=0)

In [8]:
result.head()

,title,contents
0,\n\t\t\t\t정부조직법\n\t\t\t\t\n\t\t\t,제1조(목적) 이 법은 국가행정사무의 체계적이고 능률적인 수행을 위하여 국가행정기...
1,\n\t\t\t\t학교시설사업 촉진법\n\t\t\t\t\n\t\t\t,제1조(목적) 이 법은 초등학교ㆍ중학교ㆍ고등학교 및 특수학교의 시설의 설치ㆍ이전 및...
2,\n\t\t\t\t공공차관의도입및관리에관한법률시행규칙\n\t\t\t\t,제1조(목적) 이 규칙은 공공차관의도입및관리에관한법률 및 동법시행령에서 위임된 사항...
3,\n\t\t\t\t사법정책자문위원회 규칙\n\t\t\t\t\n\t\t\t,제1조 (목적) 이 규칙은 법원조직법 제25조에 규정된 사법정책자문위원회(이하 ‘위...
4,\n\t\t\t\t연구실 안전환경 조성에 관한 법률 시행규칙\n\t\t\t\t,제1조(목적) 이 규칙은 「연구실 안전환경 조성에 관한 법률」 및 같은 법 시행령에...


### 조 별로 나누어서 나이 찾기

#### title strip

In [9]:
result['title'] = result['title'].apply(lambda x : x.strip() )

In [10]:
# index reset
result = result.reset_index(drop=True)

In [11]:
result.head()

,title,contents
0,정부조직법,제1조(목적) 이 법은 국가행정사무의 체계적이고 능률적인 수행을 위하여 국가행정기...
1,학교시설사업 촉진법,제1조(목적) 이 법은 초등학교ㆍ중학교ㆍ고등학교 및 특수학교의 시설의 설치ㆍ이전 및...
2,공공차관의도입및관리에관한법률시행규칙,제1조(목적) 이 규칙은 공공차관의도입및관리에관한법률 및 동법시행령에서 위임된 사항...
3,사법정책자문위원회 규칙,제1조 (목적) 이 규칙은 법원조직법 제25조에 규정된 사법정책자문위원회(이하 ‘위...
4,연구실 안전환경 조성에 관한 법률 시행규칙,제1조(목적) 이 규칙은 「연구실 안전환경 조성에 관한 법률」 및 같은 법 시행령에...


#### title 중복 제거

In [12]:
result = result.drop_duplicates(subset='title')
result

,title,contents
0,정부조직법,제1조(목적) 이 법은 국가행정사무의 체계적이고 능률적인 수행을 위하여 국가행정기...
1,학교시설사업 촉진법,제1조(목적) 이 법은 초등학교ㆍ중학교ㆍ고등학교 및 특수학교의 시설의 설치ㆍ이전 및...
2,공공차관의도입및관리에관한법률시행규칙,제1조(목적) 이 규칙은 공공차관의도입및관리에관한법률 및 동법시행령에서 위임된 사항...
3,사법정책자문위원회 규칙,제1조 (목적) 이 규칙은 법원조직법 제25조에 규정된 사법정책자문위원회(이하 ‘위...
4,연구실 안전환경 조성에 관한 법률 시행규칙,제1조(목적) 이 규칙은 「연구실 안전환경 조성에 관한 법률」 및 같은 법 시행령에...
...,...,...
5644,교육환경 보호에 관한 법률 시행령,제1조(목적) 이 영은 「교육환경 보호에 관한 법률」에서 위임된 사항과 그 시행에 ...
5645,과학기술정보통신부와 그 소속기관 직제,"제1조(목적) 이 영은 과학기술정보통신부와 그 소속기관의 조직과 직무범위, 그 밖..."
5646,주식ㆍ사채 등의 전자등록에 관한 법률 시행령,제1조(목적) 이 영은 「주식ㆍ사채 등의 전자등록에 관한 법률」에서 위임된 사항과 ...
5647,휴직법관에 대한 보수지급에 관한 규칙,제1조(목적 및 적용범위) 이 규칙은 「법원조직법」 제51조제2항의 규정에 의한 휴...


In [13]:
result = result.set_index('title')
result.head()

,contents
title,
정부조직법,제1조(목적) 이 법은 국가행정사무의 체계적이고 능률적인 수행을 위하여 국가행정기...
학교시설사업 촉진법,제1조(목적) 이 법은 초등학교ㆍ중학교ㆍ고등학교 및 특수학교의 시설의 설치ㆍ이전 및...
공공차관의도입및관리에관한법률시행규칙,제1조(목적) 이 규칙은 공공차관의도입및관리에관한법률 및 동법시행령에서 위임된 사항...
사법정책자문위원회 규칙,제1조 (목적) 이 규칙은 법원조직법 제25조에 규정된 사법정책자문위원회(이하 ‘위...
연구실 안전환경 조성에 관한 법률 시행규칙,제1조(목적) 이 규칙은 「연구실 안전환경 조성에 관한 법률」 및 같은 법 시행령에...


In [14]:
result.describe()

,contents
count,5033
unique,5033
top,제1조(목적) 이 영은 「자연공원법」에서 위임된 사항과 그 시행에 관하여 필요한 사...
freq,1


In [15]:
result.isnull().sum()

contents    25
dtype: int64

In [16]:
result = result.dropna() # 결측치 제거

In [17]:
len(result)

5033

#### 나이 후보군 찾기

In [18]:
result_df = pd.DataFrame()
cnt = 0
for law in tqdm(result.index): # 법령명 하나의
    for text in result.loc[law]['contents'].split('.'): # 문장 . 으로 분리 후
        
        age_find = re.findall(r'[만]*[\s]*[0-9]+[\s]*세[^대환제트관임월][\s]*[이]?[상하전후내외]?[미]?[만]?[초]?[과]?[까]?[지]?[부]?[터]?', text)
        jo_text = re.findall(r'제[0-9]+조[의]?[0-9]?\([가-힇\s]+\)', text)
        if jo_text!=[]:
            cur_jo_text = jo_text
        if age_find:
            
            if jo_text: # 같은 조에 속할때
                title = law
                jo_title = cur_jo_text[0]
                
                age_list = [re.sub('[^0-9만세이상하전후내외미만초과까지부터]+', '', i) for i in age_find] # 한 줄 안에 여러 나이가 있을때
                jo_contents = re.sub('[\[\]\>]', '', text) 
                result_df = result_df.append({
                    'title':title,
                    'jo_title':jo_title,
                    'jo_contents': jo_contents,
                    'age_list': age_list
                }, ignore_index=True)
            else: # 조 안에서 여러 줄로 나누어 져 있을때
                jo_title = cur_jo_text[0]
                jo_contents = re.sub('[\[\]\>]', '', text) 
                title = law
                age_list = [re.sub('[^A-Za-z0-9가-힣]', '', i) for i in age_find] # 한 줄 안에 여러 나이가 있을때
                result_df = result_df.append({
                    'title':title,
                    'jo_title':jo_title,
                    'jo_contents': jo_contents,
                    'age_list': age_list
                }, ignore_index=True)


In [19]:
result_df

,age_list,jo_contents,jo_title,title
0,[13세이상],"제18조의2(자의 의견의 청취) 가정법원이 미성년자인 자의 친권자 지정, 양육과 ...",제18조의2(자의 의견의 청취),가사소송규칙
1,[13세이상],제59조의2(관계자의 의견의 청취) ① 가정법원은 「민법」 제781조제5항의 규정...,제59조의2(관계자의 의견의 청취),가사소송규칙
2,[13세이상],② 가정법원은「민법」 제781조제6항의 규정에 의한 자의 성과 본의 변경허가 청...,제59조의2(관계자의 의견의 청취),가사소송규칙
3,[13세이상],제62조의3(관계자의 의견의 청취) ① 가정법원은 친양자 입양에 관한 심판을 하기...,제62조의3(관계자의 의견의 청취),가사소송규칙
4,[13세이상],④ 가정법원은 미성년후견인ㆍ미성년후견감독인의 선임과 변경 심판을 하는 경우 그 ...,제64조(친권행사방법의 결정),가사소송규칙
...,...,...,...,...
1384,[65세이상],제19조(경로우대시설의 종류 등) ①법 제26조제1항의 규정에 의하여 65세이상의...,제19조(경로우대시설의 종류 등),노인복지법 시행령
1385,[65세이상],②65세이상의 자가 경로우대시설의 이용요금을 할인하여 이용하고자 하는 때에는 당...,제19조(경로우대시설의 종류 등),노인복지법 시행령
1386,[19세미만],학교폭력 등 소년범죄: 소년(19세 미만인 사람을 말한다,제3조(수사 관련 자치경찰사무의 범위 등),자치경찰사무와 시ㆍ도자치경찰위원회의 조직 및 운영 등에 관한 규정
1387,[19세이상],"다만, 그 소년이 해당 사건에서 19세 이상인 사람과 「형법」 제30조부터 제32...",제3조(수사 관련 자치경찰사무의 범위 등),자치경찰사무와 시ㆍ도자치경찰위원회의 조직 및 운영 등에 관한 규정


In [20]:
result_df.describe()

,age_list,jo_contents,jo_title,title
count,1389,1389,1389,1389
unique,421,1333,892,458
top,[65세이상],19세 미만인 사람 2,제2조(정의),병역법 시행령
freq,81,5,49,47


In [21]:
def clean_item(d):
    d = d['age_list']
    
    return [re.sub('[가의를인에로]+', '', i) for i in d]

In [22]:
result_df['age_list'] = result_df.apply(clean_item, axis=1)

In [29]:
result_df.sample(10)

,age_list,jo_contents,jo_title,title
111,[20세이상],법 제28조제2항제10호: 별표 2의 근무ㆍ연구 경력기준 제8조(응시연령) 전문경...,제8조(응시연령),전문경력관 규칙
174,[65세이상],②국가 또는 지방자치단체는 노인의 일상생활에 관련된 사업을 경영하는 자에게 65...,제26조(경로우대),노인복지법
1228,"[51세, 52세, 56세, 57세]",중령 1) 제1차 재임용 심사 대상 연령: 51세에서 52세 사이 ...,제8조(현역정년),군인사법
9,[18세이상],개인 : 18세 이상으로서 「국가공무원법」 제33조 각 호의 어느 하나에 해당하지...,제4조(우편업무의 일부를 수탁할 수 있는 자의 자격),우편법 시행규칙
291,[50세이상],"제86조의4(연금계좌세액공제 등) 종합소득이 있으며, 해당 과세기간에 「소득세법...",제86조의4(연금계좌세액공제 등),조세특례제한법
1196,"[60세이상, 64세이하]",60세 이상 64세 이하인 사람은 부양의무자 또는 피부양자가 아닌 자활가능자로 본다,제130조(가사사정으로 인한 전시근로역 편입 및 대체역의 소집해제 등),병역법 시행령
557,[13세이상],이하 같다)을 간음하거나 13세 이상의 장애 아동ㆍ청소년으로 하여금 다른 사람을 ...,제5조(사회의 책임),아동ㆍ청소년의 성보호에 관한 법률
899,[5세미만],“안전용기ㆍ포장”이란 5세 미만 어린이가 열기 어렵게 설계ㆍ고안된 용기나 포장을 말한다,제2조(정의),약사법
85,[19세미만],19세 미만의 자녀ㆍ손자녀와 함께 입소하는 자 ⑦입소대상자가 계약을 체결할 수 ...,제15조(양로시설 등의 입소절차 등),노인복지법 시행규칙
649,[60세이상],이월공제받는 과세연도에 최초로 근로계약을 체결한 제1호 외의 상시근로자 중 청년근...,제174조(세액공제액의 이월공제),지방세특례제한법


In [35]:
result_df.to_json('data.json','records',force_ascii=False)

단순 연령대별 시각화